In [9]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 48.6 MB/s eta 0:00:00


In [4]:
!pip install langchain

  Using cached langchain-0.1.14-py3-none-any.whl (812 kB)
  Using cached dataclasses_json-0.6.4-py3-none-any.whl (28 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
  Using cached langchain_community-0.0.31-py3-none-any.whl (1.9 MB)
  Using cached langchain_core-0.1.40-py3-none-any.whl (276 kB)
  Using cached langchain_text_splitters-0.0.1-py3-none-any.whl (21 kB)
  Using cached langsmith-0.1.40-py3-none-any.whl (87 kB)
  Using cached marshmallow-3.21.1-py3-none-any.whl (49 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
  Using cached jsonpointer-2.4-py2.py3-none-any.whl (7.8 kB)
  Using cached packaging-23.2-py3-none-any.whl (53 kB)
  Using cached orjson-3.10.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (144 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [11]:
!pip install bitsandbytes

In [12]:
!pip install transformers

In [18]:
!pip install accelerate

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [2]:
model="meta-llama/Llama-2-7b-chat-hf"

In [3]:
tokenizer=AutoTokenizer.from_pretrained(model,use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:732: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [20]:
!pip install accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.38.2
Uninstalling transformers-4.38.2:
  Successfully uninstalled transformers-4.38.2
Found existing installation: accelerate 0.28.0
Uninstalling accelerate-0.28.0:
  Successfully uninstalled accelerate-0.28.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 29.6 MB/s eta 0:00:00
  Using cached accelerate-0.28.0-py3-none-any.whl (290 kB)


In [4]:
model=AutoModelForCausalLM.from_pretrained(model,
                                           device_map='auto',
                                           torch_dtype=torch.float16,
                                           use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [5]:
pipeline=transformers.pipeline("text-generation",
                               model=model,
                               tokenizer=tokenizer,
                               torch_dtype=torch.bfloat16,
                               device_map="auto",
                               # Maximum number of tokens to generate, a word is generally 2-3 tokens (minimum: 1)
                               max_new_tokens=512,
                               #Minimum number of tokens to generate. To disable this set it to -1. A word is generally 2-3 tokens (minimum 1)
                               min_new_tokens=-1,
                               #Adjusts the randomness of output
                               temperature=0.75,
                               do_sample=True,
                               top_k=30,
                               num_return_sequences=1,
                               eos_token_id=tokenizer.eos_token_id)

https://replicate.com/meta/llama-2-7b-chat

In [6]:
llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})

In [7]:
llm.predict("Please provide a concise summary of the Book Harry Potter")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


"Please provide a concise summary of the Book Harry Potter and the Philosopher's Stone by J.K. Rowling.\nHarry Potter, a young boy who has been living with his cruel and neglectful relatives, the Dursleys, discovers that he is actually a wizard and begins attending Hogwarts School of Witchcraft and Wizardry. There, he makes friends with Ron Weasley and Hermione Granger and becomes involved in a mystery surrounding the powerful Sorcerer's Stone. Along with his friends, Harry must prevent the stone from falling into the wrong hands and confront the evil wizard, Lord Voldemort, who killed his parents and seeks to return to power.\n\nPlease provide your answer in a concise summary format."

In [8]:
B_INST, E_INST= "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<<SYS>>\n\n"

In [9]:
DEFAULT_SYSTEM_PROMPT="""\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
"""

In [10]:
instruction = "Who won the T20 Cricket World Cup 2022"

In [11]:
SYSTEM_PROMPT=B_SYS + DEFAULT_SYSTEM_PROMPT + E_SYS

In [12]:
SYSTEM_PROMPT

"<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\n<<SYS>>\n\n"

In [13]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [14]:
template

"[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\n<<SYS>>\n\nWho won the T20 Cricket World Cup 2022[/INST]"

# Text Translation

In [15]:
instruction = "Convert the following text from English to French: \n\n{text}"

In [16]:
SYSTEM_PROMPT=B_SYS+DEFAULT_SYSTEM_PROMPT+E_SYS

In [17]:
SYSTEM_PROMPT

"<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\n<<SYS>>\n\n"

In [18]:
template=B_INST+SYSTEM_PROMPT+instruction+E_INST

In [19]:
template

"[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\n<<SYS>>\n\nConvert the following text from English to French: \n\n{text}[/INST]"

In [20]:
print(template)

[INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

<<SYS>>

Convert the following text from English to French: 

{text}[/INST]


In [21]:
prompt=PromptTemplate(input_variables=["text"], template=template)

In [22]:
text ="How are you"

In [23]:
LLM_Chain=LLMChain(llm=llm, prompt=prompt)

In [25]:
LLM_Chain.invoke(text)

{'text': '[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don\'t know the answer to a question, please don\'t share false information.\n\n<<SYS>>\n\nConvert the following text from English to French: \n\nHow are you[/INST]  Sure, I\'d be happy to help! Here\'s the translation of "How are you" from English to French:\n\nComment vas-tu?\n\nI hope this helps! Let me know if you have any other questions.'}

## Custom

In [26]:
instruction = "Convert the following text from English to French: \n\n{text}"

In [27]:
CUSTOM_SYSTEM_PROMPT="You are an advanced assistant that excels at translation."

In [28]:
SYSTEM_PROMPT=B_SYS+CUSTOM_SYSTEM_PROMPT+E_SYS

In [29]:
SYSTEM_PROMPT

'<<SYS>>\nYou are an advanced assistant that excels at translation.\n<<SYS>>\n\n'

In [30]:
template=B_INST+SYSTEM_PROMPT+instruction+E_INST

In [31]:
print(template)

[INST]<<SYS>>
You are an advanced assistant that excels at translation.
<<SYS>>

Convert the following text from English to French: 

{text}[/INST]


In [32]:
prompt=PromptTemplate(input_variables=["text"], template=template)

In [33]:
text ="How are you"

In [34]:
LLM_Chain=LLMChain(llm=llm, prompt=prompt)

In [39]:
print(LLM_Chain.run(text))

[INST]<<SYS>>
You are an advanced assistant that excels at translation.
<<SYS>>

Convert the following text from English to French: 

How are you[/INST]  Sure! Here is the translation of "How are you" from English to French:

Comment vas-tu?
